In [55]:
import subprocess
import re

from typing import List

create_re = re.compile(r'CREATE TABLE public\.([a-z_]+) \(\n[ ]{4}(.+\n)+')
fk_re = re.compile(r'(FOREIGN KEY (\([a-z_]+\)) REFERENCES public.[a-z_]+\([a-z_]+\))')

DTYPE_CONVERSION = [
    ('varchar', 'STRING'),
    ('character', 'STRING'),
    ('char', 'STRING'),
    ('text', 'STRING'),
    ('uuid', 'STRING(72)'),
    ('bigint', 'INT64'),
    ('int', 'INT64'),
    ('boolean', 'BOOL'),
    ('datetime', 'TIMESTAMP'),
    ('timestamp', 'TIMESTAMP'),
    ('time', 'TIMESTAMP'),
    ('date', 'TIMESTAMP'),
    ('jsonb', 'JSON')
]

def get_table_create_statement(table_name: str) -> str:
    # This string uses backticks just like your terminal command
    # It fetches the URL and passes it to pg_dump in one go
    cmd = f"pg_dump -s -t {table_name} `heroku config:get DATABASE_URL -a sfdb-staging`"
    
    try:
        # shell=True is required to interpret the backticks
        result = subprocess.run(
            cmd, 
            shell=True, 
            capture_output=True, 
            text=True, 
            check=True
        )
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"Error: {e.stderr}")
        return None

def generate_table_names(table: str) -> List[str]:
    return([
        f'`moma-dw.moma_apps.{table}`',
        f'`moma-membership.moma_import.{table}`',
        f'`moma-dw.moma_apps_staging.{table}`',
        f'`moma-apps-staging.moma_import.{table}`',
    ])


In [76]:
full_statement = get_table_create_statement('gift_membership_batch_sales')

print(full_statement)

--
-- PostgreSQL database dump
--

\restrict 7gySYyWXPrBQ9wRjicyM0qPkwFZdkNKirHduO5z76mF4RVhzNnoHFa1JPvz7Vge

-- Dumped from database version 16.9 (Ubuntu 16.9-1.pgdg20.04+1)
-- Dumped by pg_dump version 18.0 (Postgres.app)

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET transaction_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_table_access_method = heap;

--
-- Name: gift_membership_batch_sales; Type: TABLE; Schema: public; Owner: udt1bh1siv5puv
--

CREATE TABLE public.gift_membership_batch_sales (
    id bigint NOT NULL,
    gift_membership_batch_id bigint NOT NULL,
    line_item_type character varying NOT NULL,
    line_item_id bigint NOT NULL,
    created_at timestamp(6) without time

In [77]:
create_statement = re.search(
    create_re,
    full_statement
)[0]

print(create_statement)

CREATE TABLE public.gift_membership_batch_sales (
    id bigint NOT NULL,
    gift_membership_batch_id bigint NOT NULL,
    line_item_type character varying NOT NULL,
    line_item_id bigint NOT NULL,
    created_at timestamp(6) without time zone NOT NULL,
    updated_at timestamp(6) without time zone NOT NULL,
    gift_giver_first_name character varying,
    gift_giver_last_name character varying,
    gift_giver_email character varying
);



In [79]:
fk_statement = re.search(
    fk_re,
    full_statement
)

fk_statement[0]

'FOREIGN KEY (gift_membership_batch_id) REFERENCES public.gift_membership_batches(id)'